<img src="https://wherobots.com/wp-content/uploads/2023/12/Inline-Blue_Black_onWhite@3x.png" alt="Wherobots Logo" width="600">

## Electric Vehicle Charging Station Site Selection Analysis

This notebook demonstrates a workflow for identifying potential areas for new electric vehicle (EV) charging station development using WherobotsDB and WherobotsAI raster inference functionality. The workflow is based on:

* Identifying existing EV charging station infrastructure
* Proximity to retail stores as a proxy for demand, and
* Proximity to solar farms
    

Existing charging station infrastructure and retail store point of interest data is determined using public data sources, while existing solar farm infrastructure is identified using Wherobots AI raster inference. By using a machine learning model trained on satellite imagery we can identify solar farms as an input to the analysis

In [1]:
from sedona.spark import *
import os
import warnings
warnings.filterwarnings('ignore')

# specifies catalog called benchmark, the havasu catalog
# need to get from terminal
config = SedonaContext.builder() \
           .config("spark.hadoop.fs.s3a.bucket.wherobots-examples.aws.credentials.provider", "org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider") \
           .config("spark.driver.maxResultSize", "10g") \
           .config("sedona.join.autoBroadcastJoinThreshold", "-1") \
           .config("spark.sql.catalog.benchmark.type", "hadoop") \
           .config("spark.sql.catalog.benchmark", "org.apache.iceberg.spark.SparkCatalog") \
           .config("spark.sql.catalog.benchmark.warehouse", "s3://wherobots-inference-staging/benchmark/") \
           .config("spark.sql.catalog.benchmark.io-impl", "org.apache.iceberg.aws.s3.S3FileIO") \
           .config("spark.sql.catalog.benchmark.client.arn", f"os.getenviron[AWS_ROLE_ARN]") \
           .config("spark.sql.catalog.benchmark.client.region",  "us-west-2") \
           .config("spark.hadoop.fs.s3a.bucket.benchmark.arn", f"os.getenviron[AWS_ROLE_ARN]") \
           .config("spark.sql.catalog.benchmark.warehouse", "s3://wherobots-inference-staging/benchmark/").getOrCreate()
sedona = SedonaContext.create(config)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Identify Area Of Interest

We will use US Census Zip Code Tabulated Areas (ZCTA) to identify regions for potential EV charging station development. We will confine our analysis to the state of Arizona.

Note that we are using the `ST_Intersects` spatial predicate function to find ZCTAs that intersect with the border of Arizona rather than `ST_Contains`. This is because some ZCTAs extend beyond the border of Arizona and can lie within multiple states. This will extend our analysis slightly beyond the borders of Arizona.

In [2]:
az_zips_df = sedona.sql("""
WITH arizona AS ( 
    SELECT localityArea.geometry AS geometry
    FROM wherobots_open_data.overture_2024_02_15.admins_locality locality 
    JOIN wherobots_open_data.overture_2024_02_15.admins_localityArea localityArea 
    ON locality.id = localityArea.localityId
    WHERE locality.names.primary = "Arizona" AND locality.localityType = "state" 
)

SELECT ST_Intersection(arizona.geometry, zta5.geometry) AS geometry, ZCTA5CE10 
FROM wherobots_pro_data.us_census.zipcode zta5, arizona
WHERE ST_Intersects(arizona.geometry, zta5.geometry)
""")

In [3]:
az_zips_df.createOrReplaceTempView("az_zta5")

In [4]:
az_zips_df.printSchema()

root
 |-- geometry: geometry (nullable = true)
 |-- ZCTA5CE10: string (nullable = true)



In [ ]:
SedonaKepler.create_map(az_zips_df, name="Arizona ZCTAs")

Next, we will identify existing EV charging infrastructure within each ZCTA as an input to our analysis.

## Existing EV Charging Infrastructure

Using data from [Open Charge Map](https://openchargemap.org/site) we calculate the number of EV charging stations in each ZCTA to give us a sense of existing EV charging infrastructure.


In [5]:
stations_df = sedona.read.format("geoparquet").load("s3://wherobots-examples/data/examples/openchargemap/world.parquet")

In [6]:
stations_df.createOrReplaceTempView("stations")

In [ ]:
SedonaKepler.create_map(stations_df.sample(0.01), name="EV Charging Stations")

Count of existing EV charging stations per ZCTA.

In [7]:
az_stations_df = sedona.sql("""
SELECT COUNT(*) AS num, any_value(az_zta5.geometry) AS geometry, ZCTA5CE10
FROM stations JOIN az_zta5
WHERE ST_Intersects(az_zta5.geometry, stations.geometry)
GROUP BY ZCTA5CE10 
ORDER BY num DESC
""")

In [8]:
az_stations_df.createOrReplaceTempView("az_stations")

In [9]:
az_stations_df.count()

204

In [10]:
az_stations_df.cache().show()

+---+--------------------+---------+
|num|            geometry|ZCTA5CE10|
+---+--------------------+---------+
| 51|POLYGON ((-111.97...|    85281|
| 36|POLYGON ((-111.95...|    85251|
| 33|POLYGON ((-111.92...|    85260|
| 28|POLYGON ((-112.07...|    85004|
| 27|POLYGON ((-112.14...|    85226|
| 26|POLYGON ((-112.07...|    86336|
| 22|POLYGON ((-111.97...|    85054|
| 22|POLYGON ((-112.05...|    85016|
| 20|POLYGON ((-111.89...|    85286|
| 19|POLYGON ((-112.06...|    85034|
| 17|POLYGON ((-111.97...|    85254|
| 17|POLYGON ((-111.69...|    85212|
| 16|POLYGON ((-111.75...|    85206|
| 15|POLYGON ((-111.92...|    85250|
| 15|POLYGON ((-111.95...|    86001|
| 15|POLYGON ((-110.96...|    85719|
| 15|POLYGON ((-112.28...|    85305|
| 15|MULTIPOLYGON (((-...|    85282|
| 15|POLYGON ((-111.94...|    85248|
| 14|POLYGON ((-112.36...|    85323|
+---+--------------------+---------+
only showing top 20 rows



In [11]:
az_stations_df.printSchema()

root
 |-- num: long (nullable = false)
 |-- geometry: geometry (nullable = true)
 |-- ZCTA5CE10: string (nullable = true)



In [ ]:
SedonaKepler.create_map(az_stations_df)

## Arizona Retail Stores

Next, we'll use retail stores per ZCTA as a proxy for demand. Using the Overture Maps Foundation public point of interest data set.

In [12]:
sedona.table("wherobots_open_data.overture_2024_02_15.places_place").count()

53622897

In [13]:
az_retail_df = sedona.sql("""
SELECT COUNT(*) AS num, any_value(az_zta5.geometry) AS geometry, ZCTA5CE10
FROM wherobots_open_data.overture_2024_02_15.places_place places 
JOIN az_zta5
WHERE ST_Intersects(az_zta5.geometry, places.geometry)
AND places.categories.main = "retail"
GROUP BY ZCTA5CE10 
ORDER BY num DESC
""")

In [14]:
az_retail_df.createOrReplaceTempView("az_retail")

In [15]:
az_retail_df.cache().show(5)

+---+--------------------+---------+
|num|            geometry|ZCTA5CE10|
+---+--------------------+---------+
| 22|POLYGON ((-112.14...|    85226|
| 20|POLYGON ((-111.92...|    85260|
| 15|POLYGON ((-111.95...|    85251|
| 15|POLYGON ((-111.86...|    85210|
| 15|POLYGON ((-112.23...|    85308|
+---+--------------------+---------+
only showing top 5 rows



In [ ]:
SedonaKepler.create_map(az_retail_df)

## Combining Retail Stores & Existing EV Chargers

Before we apply WherobotsAI raster inference to identify solar farms in the area, we'll use existing EV chargers and retail stores to identify ZCTA with high demand and low existing EV charging infrastructure by computing the ratio of retail stores to EV chargers in each ZCTA.


In [16]:
az_ratio = sedona.sql("""
SELECT 
    coalesce(az_stations.num, 0) / coalesce(az_retail.num, 1) AS ratio, 
    coalesce(az_stations.geometry, az_retail.geometry) AS geometry, 
    coalesce(az_stations.ZCTA5CE10, az_retail.ZCTA5CE10) AS ZCTA5CE10
FROM az_retail FULL OUTER JOIN az_stations
ON az_retail.ZCTA5CE10 = az_stations.ZCTA5CE10
WHERE az_retail.num > 1
ORDER BY ratio DESC
""")

In [17]:
az_ratio.createOrReplaceTempView("az_ratio")

In [18]:
az_ratio.cache().show()

+------------------+--------------------+---------+
|             ratio|            geometry|ZCTA5CE10|
+------------------+--------------------+---------+
|              14.0|POLYGON ((-112.07...|    85004|
|              11.0|POLYGON ((-111.97...|    85054|
| 8.666666666666666|POLYGON ((-112.07...|    86336|
|               7.0|POLYGON ((-112.36...|    85323|
|               7.0|POLYGON ((-112.40...|    86023|
|               6.5|POLYGON ((-112.46...|    85395|
|               5.5|POLYGON ((-112.08...|    85003|
|               5.1|POLYGON ((-111.97...|    85281|
|               5.0|POLYGON ((-111.66...|    86004|
|               5.0|POLYGON ((-112.03...|    85008|
|               5.0|POLYGON ((-111.94...|    85248|
| 4.666666666666667|POLYGON ((-112.06...|    85014|
|              4.25|POLYGON ((-111.69...|    85212|
|               4.0|POLYGON ((-111.96...|    85257|
|              3.75|POLYGON ((-111.92...|    85250|
|               3.5|POLYGON ((-112.07...|    85012|
|3.333333333

In [ ]:
SedonaKepler.create_map(az_ratio)

ZCTAs with a low "ratio" are potential candidates for additional EV charging stations. The final input to our analysis is proximity to solar farms, which we will identify using WherobotsAI raster inference.

## WherobotsAI Raster Inference

The [outdb raster table](https://docs.wherobots.com/1.2.2/references/havasu/raster/out-db-rasters/) refers to Sentinel-2 images with low cloud cover during 2023 in Arizona. We've prepared this using WherbotsDB's raster processing capabilities.

TODO: identify solar farms within ZCTAs, prioritize low "ratio" ZCTAs


In [19]:
columns_to_drop = ["x", "y", "product_type", "length"]
num_partitions = 32
solar_model_inputs_df = sedona.table("benchmark.db.solar_satlas_sentinel2_db").drop(*columns_to_drop).repartition(num_partitions)

In [20]:
solar_model_inputs_df.cache().show()

+--------------------+-------------+---------+-------------+--------------------+--------------+------------+--------------------+--------------------+
|            filename|surrogate_key|grid_code|constellation|            geometry|start_datetime|end_datetime|                path|        outdb_raster|
+--------------------+-------------+---------+-------------+--------------------+--------------+------------+--------------------+--------------------+
|sentinel-2-S2MSI2...|   1003289323|    12SUD|   sentinel-2|POLYGON ((-113.19...|      20230107|    20230206|s3a://wherobots-i...|OutDbGridCoverage...|
|sentinel-2-S2MSI2...|  -1857201540|    12SWA|   sentinel-2|POLYGON ((-111.00...|      20231001|    20231026|s3a://wherobots-i...|OutDbGridCoverage...|
|sentinel-2-S2MSI2...|   1090044652|    12SXD|   sentinel-2|POLYGON ((-109.90...|      20230409|    20230504|s3a://wherobots-i...|OutDbGridCoverage...|
|sentinel-2-S2MSI2...|   -638119537|    12SXB|   sentinel-2|POLYGON ((-109.64...|      2

In [21]:
az_high_demand_with_scene_geom = sedona.sql(""" 
    WITH base as (
        SELECT s.filename, s.geometry as scene_geometry, s.outdb_raster as
        outdb_raster, z.ZCTA5CE10 as zip_code_name, z.geometry as zip_geometry, z.ratio as ratio
        FROM benchmark.db.solar_satlas_sentinel2_db s, az_ratio z
        WHERE ST_Intersects(s.geometry, z.geometry)
        AND z.ratio < 1 AND start_datetime > 20231001
    )
    SELECT DISTINCT filename, outdb_raster from base""").repartition(num_partitions)

In [22]:
%%time
print(az_high_demand_with_scene_geom.cache().count())

2200
CPU times: user 11.5 ms, sys: 2.05 ms, total: 13.5 ms
Wall time: 8.09 s


In [23]:
az_high_demand_with_scene_geom.createOrReplaceTempView("az_high_demand_with_scene")

In [24]:
model_id = 'solar-satlas-sentinel2'

sedona.sql(f"""
CREATE OR REPLACE TEMP VIEW segment_fields AS (
    SELECT
        outdb_raster, 
        RS_SEGMENT('{model_id}', outdb_raster) AS segment_result
    FROM
    az_high_demand_with_scene
)
""")

DataFrame[]

In [25]:
predictions_df = sedona.sql(f"""
SELECT
  outdb_raster, segment_result.*
FROM segment_fields
""")

In [26]:
%%time
print(predictions_df.cache().count())

2200
CPU times: user 26 ms, sys: 22.3 ms, total: 48.3 ms
Wall time: 4min 6s


In [27]:
predictions_df.show()
predictions_df.createOrReplaceTempView("predictions_df")

+--------------------+--------------------+-----------------+
|        outdb_raster|    confidence_array|        class_map|
+--------------------+--------------------+-----------------+
|OutDbGridCoverage...|[0.50000113, 0.50...|{Solar Farm -> 1}|
|OutDbGridCoverage...|[0.5000018, 0.500...|{Solar Farm -> 1}|
|OutDbGridCoverage...|[0.50000083, 0.50...|{Solar Farm -> 1}|
|OutDbGridCoverage...|[0.5000002, 0.5, ...|{Solar Farm -> 1}|
|OutDbGridCoverage...|[0.5000053, 0.500...|{Solar Farm -> 1}|
|OutDbGridCoverage...|[0.50004023, 0.50...|{Solar Farm -> 1}|
|OutDbGridCoverage...|[0.5000153, 0.500...|{Solar Farm -> 1}|
|OutDbGridCoverage...|[0.50001156, 0.50...|{Solar Farm -> 1}|
|OutDbGridCoverage...|[0.50000346, 0.50...|{Solar Farm -> 1}|
|OutDbGridCoverage...|[0.500004, 0.5000...|{Solar Farm -> 1}|
|OutDbGridCoverage...|[0.50002456, 0.50...|{Solar Farm -> 1}|
|OutDbGridCoverage...|[0.5000042, 0.500...|{Solar Farm -> 1}|
|OutDbGridCoverage...|[0.5000466, 0.500...|{Solar Farm -> 1}|
|OutDbGr

In [ ]:
predictions_polys_df = sedona.sql("""
    WITH t AS (
        SELECT RS_SEGMENT_TO_GEOMS(outdb_raster, confidence_array, array(1), class_map, 0.65) result
        FROM predictions_df
    )
    SELECT result.* FROM t
""")

In [ ]:
#df_multipolys.show()

In [ ]:
predictions_polys_df.createOrReplaceTempView("predictions_polys")

predictions_polys_df = sedona.sql("""
    SELECT
        class_name[0] AS class, average_pixel_confidence_score[0] AS avg_confidence_score, ST_SetSRID(ST_Collect(geometry), 4326) AS geometry
    FROM
        predictions_polys
""").filter("ST_IsEmpty(geometry) = False")

In [ ]:
predictions_polys_df.cache().count()

In [ ]:
predictions_polys_df.createOrReplaceTempView("predictions_polys")

In [ ]:
predictions_polys_df.show()

In [ ]:
SedonaKepler.create_map(predictions_polys_df, name="Detected Solar Farms")

In [ ]:
az_solar_zip_codes = sedona.sql("""
SELECT ST_AreaSpheroid(ST_Union_Aggr(ST_SetSRID(predictions_polys.geometry, 4326))) / 1000000 * 247.10559991919519 AS solar_area, any_value(az_zta5.geometry) AS geometry, ZCTA5CE10
FROM predictions_polys JOIN az_zta5
WHERE ST_Intersects(az_zta5.geometry, predictions_polys.geometry)
GROUP BY ZCTA5CE10 
ORDER BY solar_area DESC
""")

In [ ]:
az_solar_zip_codes.show()

In [ ]:
az_solar_zip_codes.count()

In [ ]:
az_solar_zip_codes.createOrReplaceTempView("az_solar_zip_codes")

In [ ]:
SedonaKepler.create_map(az_solar_zip_codes)

In [ ]:
sedona.sql("""

SELECT MIN(ratio), MAX(ratio) 
FROM az_ratio
""").show()

#(max - min )/ max

In [ ]:
# join az_ratio with az_solar_zip codes

final_az_scores = sedona.sql("""
WITH min_max AS (
  SELECT MIN(ratio) AS ratio_min, MAX(ratio) AS ratio_max
  FROM az_ratio
)


SELECT 
    (solar_area / (ST_AreaSpheroid(az_solar_zip_codes.geometry)/ 1000000 * 247.10559991919519)) + (1 - ( (ratio - 0.0 ) / (14.0 - 0)   )) AS score, 
    (1 - (ratio / 14)) AS temp_score,
    ratio,
    az_solar_zip_codes.ZCTA5CE10 AS ZCTA5CE10,
    az_solar_zip_codes.geometry AS geometry
FROM az_ratio
JOIN az_solar_zip_codes
WHERE az_ratio.ZCTA5CE10 = az_solar_zip_codes.ZCTA5CE10
ORDER BY score DESC
""")

In [ ]:
final_az_scores.cache().show()

In [ ]:
final_az_scores.createOrReplaceTempView("final_scores")

In [ ]:
final_map = SedonaKepler.create_map(final_az_scores, "EV Site Selection Scores")
#SedonaKepler.add_df(final_map, stations_df, "Existing EV Charging stations")
final_map

In [ ]:
# Find ev chargers in priority areas

final_az_chargers = sedona.sql("""
SELECT stations.geometry, stations.id AS station_id
FROM stations, final_scores
WHERE ST_Contains(final_scores.geometry, stations.geometry)
""")

final_az_chargers.cache().count()

In [ ]:
# Find all solar farms in final 

## Write Analysis Results AS PMTiles

In [ ]:
# write final_az_scores to vector tiles
# write existing_ev chargers to vector tiles
# write retail stores to vector tiles


from wherobots import vtiles
from pyspark.sql.functions import lit


zip_tiles_path = os.getenv("USER_S3_PATH") + "final_az_suitability.pmtiles"
ev_chargers_path = os.getenv("USER_S3_PATH") + "final_az_chargers.pmtiles"

final_az_scores_layers = final_az_scores.withColumn("layer", lit('Suitability Results'))
final_az_chargers_layers = final_az_chargers.withColumn("layer", lit('Existing Chargers'))

#tiles_df = vtiles.generate(final_az_scores_layers)
#tiles_df.write.mode("overwrite").format("parquet").save(os.getenv("USER_S3_PATH") + "/tiles.parquet")

#final_az_chargers_layers.write.mode("overwrite").format("parquet").save(os.getenv("USER_S3_PATH") + "/tiles.parquet")

#vtiles.write_pmtiles(tiles_df, zip_tiles_path, features_df=final_az_scores_layers)

charger_tiles_df = vtiles.generate(final_az_chargers_layers)
charger_tiles_df.write.mode("overwrite").format("parquet").save(os.getenv("USER_S3_PATH") + "/charger_tiles.parquet")

#vtiles.write_pmtiles(charger_tiles_df, ev_chargers_path, features_df=final_az_chargers_layers)

#vtiles.show_pmtiles([zip_tiles_path, ev_chargers_path])

#az_zips_df = az_zips_df.withColumn('layer', lit('Layer 1'))
#zip_tiles_df = vtiles.generate(az_zips_df)
#vtiles.write_pmtiles(zip_tiles_df, zip_tiles_path, features_df=az_zips_df)

#vtiles.show_pmtiles(zip_tiles_path)

In [ ]:
az_high_demand_df = sedona.sql("""
FROM wherobots_open_data.overture_2024_02_15.places_place places 
JOIN az_zta5
WHERE ST_Intersects(az_zta5.geometry, places.geometry)
AND places.categories.main = "retail"
GROUP BY ZCTA5CE10 
ORDER BY num DESC
""")